# Vector Search with Cypher

Cypher is Neo4j's graph query language that allows users to store and retrieve data from the graph database. It is a declarative language, meaning that the user specifies what they want to retrieve, but not how to retrieve it. Cypher is a relatively simple language to learn, and is very powerful for querying graph data.

## Imports

In [2]:
from dotenv import load_dotenv
import os

from langchain_community.graphs import Neo4jGraph

## Set up Neo4j

In [3]:
# Load from environment
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [6]:
# Create a knowledge graph using Langchain's Neo4j integration.
# This will be used for direct querying of the knowledge graph. 
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

## Vector search with Cypher



In [18]:
## Create a vector index for movie descriptions
kg.query("""
  CREATE VECTOR INDEX movie_tagline_embeddings IF NOT EXISTS
  FOR (m:Movie) ON (m.taglineEmbedding) 
  OPTIONS { indexConfig: {
    `vector.dimensions`: 1536,
    `vector.similarity_function`: 'cosine'
  }}"""
)


[]

In [ ]:
# Add embeddings to the vector index by calling to OpenAI API directly from Cypher
# 
# MATCH a single node pattern of nodes labeled `Movie`
# WITH those movies and a function call to encode the tagline property using OpenAI
# CALL db.create.setNodeVectorProperty to set the `taglineEmbedding` property on the movie node 
kg.query("""
    MATCH (movie:Movie) WHERE movie.tagline IS NOT NULL
    WITH movie, genai.vector.encode(movie.tagline, "OpenAI", {token: $openAiApiKey}) AS vector
    CALL db.create.setNodeVectorProperty(movie, "taglineEmbedding", vector)
    """, 
    params={"openAiApiKey":OPENAI_API_KEY} )

In [20]:
question = "What movies are about love?"
kg.query("""
    WITH genai.vector.encode($question, "OpenAI", {token: $openAiApiKey}) AS embedding
    CALL db.index.vector.queryNodes('movie_tagline_embeddings', $top_k, embedding) YIELD node AS movie, score
    RETURN movie.title + " -- " + movie.tagline AS result
    """, 
    params={"openAiApiKey":OPENAI_API_KEY,
            "question": question,
            "top_k": 5
            })

Failed to write data to connection ResolvedIPv4Address(('127.0.0.1', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))
Failed to write data to connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))
Unable to retrieve routing information


ServiceUnavailable: Unable to retrieve routing information